In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from tpo_tools.instrumentation import RigolDHO1074, RigolDG2102, AWGSettings, AWGChannelSettings, OscilloscopeChannelSettings, OscilloscopeSettings, SampleInfo
from tpo_tools.analysis import PUNDTemplateWF, TrapezoidTemplateWF, CollectionTemplateWF
import pyvisa
from pprint import pprint

In [9]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())

available_devices = {}
for address in rm.list_resources() + ('TCPIP0::10.97.108.250::inst0::INSTR',):
    print()
    try:
        with rm.open_resource(address) as session:
            print(session)
            idn = session.query('*IDN?').strip()
            print(idn)
            if 'DG2102' in idn:
                available_devices['AWG'] = address
            elif 'DHO1074' in idn:
                available_devices['oscope'] = address
            elif 'CA5351' in idn:
                available_devices['TIA'] = address

    except:
        print('Failed to connect', address)
print()
pprint(available_devices)

('TCPIP0::10.97.108.216::inst0::INSTR', 'TCPIP0::0.0.0.0::inst0::INSTR', 'ASRL3::INSTR', 'ASRL7::INSTR')

TCPIPInstrument at TCPIP0::10.97.108.216::inst0::INSTR
Rigol Technologies,DG2102,DG2P233800951,00.02.07.00.01

Failed to connect TCPIP0::0.0.0.0::inst0::INSTR

SerialInstrument at ASRL3::INSTR
Failed to connect ASRL3::INSTR

SerialInstrument at ASRL7::INSTR
NF Corporation,CA5351,9355286,1.10

TCPIPInstrument at TCPIP0::10.97.108.250::inst0::INSTR
RIGOL TECHNOLOGIES,DHO1074,HDO1B244902222,00.02.13

{'AWG': 'TCPIP0::10.97.108.216::inst0::INSTR',
 'TIA': 'ASRL7::INSTR',
 'oscope': 'TCPIP0::10.97.108.250::inst0::INSTR'}


In [ ]:
awg = RigolDG2102.identify(available_devices['AWG'])
oscope = RigolDHO1074.identify(available_devices['oscope'])

with rm.open_resource(available_devices['TIA']) as tia:
    tia.write(':INPUT ON')
    # ON means the amp is atually grounded to real GND
    print(tia.query(':INPUT?'))

At address TCPIP0::10.97.108.216::inst0::INSTR, RigolDG2102 query result: Rigol Technologies,DG2102,DG2P233800951,00.02.07.00.01
At address TCPIP0::10.97.108.250::inst0::INSTR, RigolDHO1074 query result: RIGOL TECHNOLOGIES,DHO1074,HDO1B244902222,00.02.13
1



In [ ]:
# Set up the system
#